In [1]:
import os
os.getcwd()
os.chdir("..")
os.getcwd()


'/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System'

In [3]:
%%capture
!pip install -r requirements.txt

In [5]:

from unstructured.partition.pdf import partition_pdf

In [6]:
elements = partition_pdf(
    filename = "data/2015-cocacola-10k-filing.pdf",
    infer_table_structure =True,
    extract_images_from_pdf=False
)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [8]:
category_counts = {}

for element in elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.Title'>": 375,
 "<class 'unstructured.documents.elements.NarrativeText'>": 1249,
 "<class 'unstructured.documents.elements.Text'>": 751,
 "<class 'unstructured.documents.elements.Image'>": 9,
 "<class 'unstructured.documents.elements.Table'>": 114,
 "<class 'unstructured.documents.elements.Footer'>": 183,
 "<class 'unstructured.documents.elements.ListItem'>": 325,
 "<class 'unstructured.documents.elements.FigureCaption'>": 6,
 "<class 'unstructured.documents.elements.Header'>": 16}

In [45]:
categorized_elements=[]
from typing import Any

from pydantic import BaseModel
class Element(BaseModel):
    type: str
    text: Any
    metadata: dict


for element in elements:
    md = element.metadata.to_dict() if hasattr(element, "metadata") else {}
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(
            Element(type="table", text=str(element), metadata=md)
        )
    else:
        categorized_elements.append(
            Element(type="text", text=str(element), metadata=md)
        )

In [77]:
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements)) 

114
2914


In [133]:
def add_metadata(elements):
    current_section="unknown"
    for el in elements:
        # Convert metadata object to dict
        md = el.metadata
        
        # Extract filename safely
        filename = md.get("filename", "")
        
        # Find 4-digit year in filename
        match = re.search(r'(\d{4})', filename)
        if match:
            el.metadata['year'] = int(match.group(1))   # add year to metadata
            el.metadata['filing'] = "10-K"              # example: add filing type
        else:
            el.metadata.year = None
            el.metadata.filing = "10-K"
        text = el.text.replace("\n"," ").replace("  "," ")
        match = re.search(r"(ITEM\s+\d+[A-Z]?\.\s+[A-Z\s,&\-]+)", text)
        if match:
            current_section = match.group(0).strip()
            el.metadata['section']=current_section
            continue
        el.metadata['section']=current_section
    
    return elements

            
    
                  
            

In [135]:
tables =   add_metadata(table_elements)
texts = add_metadata(text_elements)

In [137]:
for el in tables:   # iterate element by element  # show first 100 chars
    print(el.metadata)  # show metadata dictionary
    print("----")

{'detection_class_prob': 0.7172439098358154, 'coordinates': {'points': ((76.26579284667969, 173.65541076660156), (76.26579284667969, 1062.1529541015625), (1610.506103515625, 1062.1529541015625), (1610.506103515625, 173.65541076660156)), 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}, 'last_modified': '2025-05-23T15:02:54', 'text_as_html': '<table><tbody><tr><td></td><td></td><td>Page</td></tr><tr><td></td><td>Forward-Looking Statements</td><td></td></tr><tr><td>art I</td><td></td><td></td></tr><tr><td>tem 1.</td><td>Business</td><td></td></tr><tr><td>tem 1A.</td><td>Risk Factors</td><td></td></tr><tr><td>tem 1B.</td><td>Unresolved Staff Comments</td><td></td></tr><tr><td>tem 2.</td><td>Properti</td><td></td></tr><tr><td>tem 3.</td><td>Legal Proceedings</td><td></td></tr><tr><td>tem 4.</td><td>Mine Safety Disclosures</td><td>BREBREBE</td></tr><tr><td>art IT</td><td></td><td></td></tr><tr><td></td><td>Market for Registrant\'s Common Equity, Related Stockholder Matte

In [139]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM


In [141]:
prompt_text = """You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = OllamaLLM(model ='llama3.2:latest')
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [143]:
tables = [i.text for i in tables]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [ ]:
texts = [i.text for i in texts]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OllamaEmbeddings(model="bge-m3:latest"))

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM
# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = OllamaLLM(model="llama3.2:latest")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What was Coca-Cola’s net cash from operating activities in 2014?")

In [ ]:
retriever.invoke("What was Coca-Cola’s net cash from operating activities in 2014?")